In [110]:
import re
from sklearn.metrics import confusion_matrix, accuracy_score

In [20]:
INPUT_PATH = './dataset/'
OUTPUT_PATH = './results/'

In [15]:
from openai import OpenAI
KEY = "API_KEY"

def chat(question):
    client = OpenAI(
        # This is the default and can be omitted
        # api_key=key
    )
    completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "You are a professional lawyer and want to know the legislature bill number and passed year."},
        {"role": "user", "content": f"{question}"}
      ],
      temperature=0
    )
    ans = completion.choices[0].message.content
    return(ans)

In [16]:
test_question = """
when did Texas pass legislation mandating the duties of a law enforcement agency regarding missing children and missing persons and to the duties of a justice of the peace or other investigator regarding unidentified bodies? The answer should be formatted as:
Year: ,Bill:
"""

In [17]:
chat(test_question)

'Year: 2019, Bill: HB 417'

In [18]:
chat(test_question)

'Year: 2019, Bill: HB 417'

In [19]:
import pandas as pd

In [73]:
def get_results(data_file_name):
    verb_list = ['add', 'amend', 'repeal']
    df = pd.read_csv(INPUT_PATH + data_file_name)
    
    df_pass = df[df['status_desc'] == 'Passed']
    print(df_pass.shape[0])
    df_pass['verb'] = df_pass['description'].str.split(' ').str[3]
    df_pass['ed'] = df_pass['description'].str.split(' ').str[3] + 'ed' 
    df_pass['remain_description'] = df_pass['description'].str.split(' ').str[4:]
    df_pass['remain_description'] = df_pass['remain_description'].apply(lambda x: ' '.join(x))
    df_pass['question'] = df_pass['ed'] + ' ' + df_pass['remain_description']
    
    df_pass['year'] = pd.to_datetime(df_pass['status_date']).dt.year
    
    df_verb = df_pass[df_pass['verb'].isin(verb_list)]
    print(df_verb.shape[0])
    
    return(df_verb)

In [114]:
data_file_name = 'CA_2009-10_bills.csv'
CA_2009_10_df = get_results(data_file_name)

1772
1407


/var/folders/v0/_1hvm2l96zq34fhj1jk036q40000gp/T/ipykernel_80436/1795481887.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pass['verb'] = df_pass['description'].str.split(' ').str[3]
/var/folders/v0/_1hvm2l96zq34fhj1jk036q40000gp/T/ipykernel_80436/1795481887.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pass['ed'] = df_pass['description'].str.split(' ').str[3] + 'ed'
/var/folders/v0/_1hvm2l96zq34fhj1jk036q40000gp/T/ipykernel_80436/1795481887.py:9: SettingWithCopyWarning: 
A value is trying 

In [115]:
CA_2009_10_df.to_csv(OUTPUT_PATH + 'test_df.csv')
CA_2009_10_df

,bill_id,session_id,bill_number,status,status_desc,status_date,title,description,committee_id,committee,last_action_date,last_action,url,state_link,verb,ed,remain_description,question,year
4,59873,30,AB5,4,Passed,2009-06-29,Civil discovery: Electronic Discovery Act.,"An act to amend Sections 2016.020, 2031.010, 2...",0,NaN,2009-06-29,"Chaptered by Secretary of State - Chapter 5, S...",https://legiscan.com/CA/bill/AB5/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...,amend,amended,"Sections 2016.020, 2031.010, 2031.020, 2031.03...","amended Sections 2016.020, 2031.010, 2031.020,...",2009
8,59877,30,AB9,4,Passed,2009-10-11,Political Reform Act of 1974: expenditures.,An act to amend Sections 82015 and 82031 of th...,0,NaN,2009-10-11,"Chaptered by Secretary of State - Chapter 363,...",https://legiscan.com/CA/bill/AB9/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...,amend,amended,Sections 82015 and 82031 of the Government Cod...,amended Sections 82015 and 82031 of the Govern...,2009
11,59879,30,AB11,4,Passed,2009-10-11,Corporate reorganization: built-in losses.,An act to add Section 24458 to the Revenue and...,0,NaN,2009-10-11,"Chaptered by Secretary of State - Chapter 401,...",https://legiscan.com/CA/bill/AB11/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...,add,added,Section 24458 to the Revenue and Taxation Code...,added Section 24458 to the Revenue and Taxatio...,2009
13,59880,30,AB12,4,Passed,2010-09-30,California Fostering Connections to Success Act.,An act to amend Section 17552 of the Family Co...,0,NaN,2010-09-30,"Chaptered by Secretary of State - Chapter 559,...",https://legiscan.com/CA/bill/AB12/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...,amend,amended,"Section 17552 of the Family Code, to amend Sec...","amended Section 17552 of the Family Code, to a...",2010
17,59882,30,AB14,4,Passed,2009-10-11,Vehicles: nuisance abatement: impoundment.,An act to repeal and add Section 22659.5 of th...,0,NaN,2009-10-11,"Chaptered by Secretary of State - Chapter 210,...",https://legiscan.com/CA/bill/AB14/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...,repeal,repealed,"and add Section 22659.5 of the Vehicle Code, r...",repealed and add Section 22659.5 of the Vehicl...,2009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5040,76587,30,SB1490,4,Passed,2010-09-24,Accountants.,"An act to amend Sections 5092, 5094.6, and 509...",0,NaN,2010-09-24,"Chaptered by Secretary of State. Chapter 298, ...",https://legiscan.com/CA/bill/SB1490/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...,amend,amended,"Sections 5092, 5094.6, and 5096.12 of the Busi...","amended Sections 5092, 5094.6, and 5096.12 of ...",2010
5041,76588,30,SB1491,4,Passed,2010-09-28,Professions and vocations.,"An act to amend Sections 5020, 5021, 5024, 507...",0,NaN,2010-09-28,"Chaptered by Secretary of State. Chapter 415, ...",https://legiscan.com/CA/bill/SB1491/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...,amend,amended,"Sections 5020, 5021, 5024, 5076, 5090, 5109, 5...","amended Sections 5020, 5021, 5024, 5076, 5090,...",2010
5042,76886,30,SB1492,4,Passed,2010-09-29,Income taxes: voluntary disclosure agreements.,An act to amend Sections 19191 and 19194 of th...,0,NaN,2010-09-29,"Chaptered by Secretary of State. Chapter 498, ...",https://legiscan.com/CA/bill/SB1492/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...,amend,amended,Sections 19191 and 19194 of the Revenue and Ta...,amended Sections 19191 and 19194 of the Revenu...,2010
5043,76887,30,SB1493,4,Passed,2010-08-23,Property taxation assessor: tax collector:,"An act to amend Sections 75.31, 155.20, 465, 6...",0,NaN,2010-08-23,"Chaptered by Secretary of State. Chapter 185, ...",https://legiscan.com/CA/bill/SB1493/2009,http://www.leginfo.ca.gov/cgi-bin/postquery?bi...,amend,amended,"Sections 75.31, 155.20, 465, 619, 621, and 369...","amended Sections 75.31, 155.20, 465, 619, 621,...",2010


In [77]:
question_pattern = "When did {state} pass legislation that {description}? The answer should be formatted as: Year: ,Bill:"

In [93]:
bill_gt_list = []
year_gt_list = []
bill_predict_list = []
year_predict_list = []
for row_idx, row in CA_2009_10_df[:3].iterrows():
    print('-----------------------')
    bill_numbers = re.findall(r'\d+', row['bill_number'])[0]
    bill_letters = re.findall(r'[a-zA-Z]+', row['bill_number'])[0]
    # bill_gt = row['bill_number'][:2] + ' ' + row['bill_number'][2:]
    bill_gt = bill_letters + ' ' + bill_numbers
    year_gt = str(row['year'])
    

    description = row['question']
    question = question_pattern.format(state='California', description=description)
    print(question)
    ans = chat(question)
    # ans = 'Year: 2019, Bill: HB 417'
    year_predict = ans.split(',')[0].split('Year: ')[1]
    bill_predict = ans.split('Bill: ')[1]
    print(f"bill gt = {bill_gt}, bill predicted = {bill_predict}")
    print(f"year gt = {year_gt}, year predicted = {year_predict}")
    

-----------------------
When did California pass legislation that amended Sections 2016.020, 2031.010, 2031.020, 2031.030, 2031.040, 2031.050, 2031.060, 2031.210, 2031.220, 2031.230, 2031.240, 2031.250, 2031.260, 2031.270, 2031.280, 2031.290, 2031.300, 2031.310, and 2031.320 of, and to add Sections 1985.8 and 2031.285 to, the Code of Civil Procedure, relating to civil discovery, and declaring the urgency thereof, to take effect immediately.? The answer should be formatted as: Year: ,Bill:
bill gt = AB 5, bill predicted = AB 5
year gt = 2009, year predicted = 2019
-----------------------
When did California pass legislation that amended Sections 82015 and 82031 of the Government Code, relating to the Political Reform Act of 1974.? The answer should be formatted as: Year: ,Bill:
bill gt = AB 9, bill predicted = AB 571
year gt = 2009, year predicted = 2019
-----------------------
When did California pass legislation that added Section 24458 to the Revenue and Taxation Code, relating to ta

In [113]:
def generate_gpt_result(test_df, printout = False):
    bill_gt_list = []
    year_gt_list = []
    bill_predict_list = []
    year_predict_list = []
    question_list = []
    for row_idx, row in test_df.iterrows():
        
        # GT
        bill_numbers = re.findall(r'\d+', row['bill_number'])[0]
        bill_letters = re.findall(r'[a-zA-Z]+', row['bill_number'])[0]
        bill_gt = bill_letters + ' ' + bill_numbers
        year_gt = str(row['year'])

        
        # chatGPT
        description = row['question']
        question = question_pattern.format(state='California', description=description)
        ans = chat(question)
        # ans = 'Year: 2019, Bill: HB 417'
        
        year_predict = ans.split(',')[0].split('Year: ')[1]
        bill_predict = ans.split('Bill: ')[1]
        
        # store
        bill_gt_list.append(bill_gt)
        year_gt_list.append(year_gt)
        bill_predict_list.append(bill_predict)
        year_predict_list.append(year_predict)
        question_list.append(question)
        
        # print
        if printout:
            print('-----------------------')
            print(question)
            print(f"bill gt = {bill_gt}, bill predicted = {bill_predict}")
            print(f"year gt = {year_gt}, year predicted = {year_predict}")
    output_df = pd.DataFrame({
        'bill_gt': bill_gt_list,
        'year_gt': year_gt_list,
        'bill_predict': bill_predict_list,
        'year_predict': year_predict_list,
        'question': question_list
    })
    return(output_df)

In [119]:

test_df = CA_2009_10_df.sample(n=100)
test_df.to_csv(OUTPUT_PATH + 'CA_2009_10_sample.csv')
output_df = generate_gpt_result(test_df)
output_df.to_csv(OUTPUT_PATH + 'CA_2009_10_sample_results.csv')

In [120]:
output_df

,bill_gt,year_gt,bill_predict,year_predict,question
0,SBX 36,2009,AB 74,2019,When did California pass legislation that amen...
1,AB 66,2009,AB 1505,2019,When did California pass legislation that amen...
2,SBX 51,2010,AB 1505,2019,When did California pass legislation that amen...
3,AB 2510,2010,AB 340,2012,When did California pass legislation that adde...
4,AB 428,2009,AB 1331,2018,When did California pass legislation that amen...
...,...,...,...,...,...
95,AB 2302,2010,AB 1313,2019,When did California pass legislation that adde...
96,AB 955,2009,AB 392,2019,When did California pass legislation that amen...
97,AB 1514,2010,AB 571,2019,When did California pass legislation that amen...
98,AB 729,2009,AB 456,2019,When did California pass legislation that amen...


In [121]:
accuracy_score(output_df['bill_gt'].to_list(), output_df['bill_predict'].to_list())

0.02

In [122]:
accuracy_score(output_df['year_gt'].to_list(), output_df['year_predict'].to_list())

0.03